<a href="https://colab.research.google.com/github/Eternity402/Cylab-Diary/blob/master/resnet_DIY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization, AveragePooling2D, ReLU
from tensorflow.keras import Model
tf.config.experimental_run_functions_eagerly(True)

     |████████████████████████████████| 380.5MB 43kB/s 
     |████████████████████████████████| 4.3MB 48.2MB/s 
     |████████████████████████████████| 501kB 47.5MB/s 


In [0]:
# Dataset - CIFAR100
cifar100 = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Shuffle and make batch
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(50000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

169009152/169001437 [==============================] - 4s 0us/step


In [0]:
EPOCHS = 50
hist = np.empty(shape=(2,EPOCHS))
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(64, (3,3), strides = (1,1), padding = 'same', activation='linear',input_shape=x_train.shape[1:])
    self.bn1 = BatchNormalization()
    self.RL = ReLU()
    self.avg = AveragePooling2D((4,4))
    self.flatten = Flatten()
    self.FC = Dense(100, activation='softmax')

  def make_BB(self, x,num_filter, size_decrease):
    if size_decrease == True:
      C1 = Conv2D(num_filter, (3,3),padding = 'same',strides = (2,2),activation = 'linear')
      B1 = BatchNormalization()
      R1 = ReLU()
      C2 = Conv2D(num_filter, (3,3),padding = 'same',strides = (1,1),activation = 'linear')
      B2 = BatchNormalization()
      C3 = Conv2D(num_filter, (1,1),padding= 'same', strides = (2,2),activation = 'linear')
      B3 = BatchNormalization()
      forward = B2(C2(R1(B1(C1(x))))) + B3(C3(x))
      return forward
    else:
      C1 = Conv2D(num_filter, (3,3),padding = 'same',strides = (1,1),activation = 'linear')
      B1 = BatchNormalization()
      R1 = ReLU()
      C2 = Conv2D(num_filter, (3,3),padding = 'same',strides = (1,1),activation = 'linear')
      B2 = BatchNormalization()
      forward = B2(C2(R1(B1(C1(x))))) + x
      return forward


  def call(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.RL(x)
    # BasicBlock1
    x = self.make_BB(x,64,False)
    x = self.make_BB(x,64,False)
    # BasicBlock2
    x = self.make_BB(x,128,True)
    x = self.make_BB(x,128,False)
    # BasicBlock3
    x = self.make_BB(x,256,True)
    x = self.make_BB(x,256,False)
    # BasicBlock4
    x = self.make_BB(x,512,True)
    x = self.make_BB(x,512,False)
    x = self.avg(x)
    x = self.flatten(x)
    return self.FC(x)
  
  def func(self):
    x = tf.keras.layers.Input(shape=(32, 32, 3))
    return Model(inputs=[x], outputs=self.call(x))

    
        
      
model = MyModel()

# Show model's summary
# model_functional = model.func()
# model_functional.summary()

# Load saved weights
# Drive mount is needed in Colab.
model.load_weights('drive/My Drive/Colab Notebooks/AJ/check')

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(labels, predictions)

#@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)
  #model.summary()

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  hist[:,epoch] = (train_accuracy.result(),test_accuracy.result())
  print (template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  model.save_weights('drive/My Drive/Colab Notebooks/AJ/check')
  print('Weights save complete')
plt.plot(range(EPOCHS),hist[0])
plt.plot(range(EPOCHS),hist[1])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.title('result')
plt.show()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


KeyboardInterrupt: ignored

In [0]:
EPOCHS = 50
hist = np.empty(shape=(2,EPOCHS))
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(64, (3,3), strides = (1,1), padding = 'same', activation='linear',input_shape=x_train.shape[1:])
    self.bn1 = BatchNormalization()
    self.RL = ReLU()
    self.avg = AveragePooling2D((4,4))
    self.flatten = Flatten()
    self.FC = Dense(100, activation='softmax')

  def make_BB(self, x,num_filter, size_decrease):
    if size_decrease == True:
      C1 = Conv2D(num_filter, (3,3),padding = 'same',strides = (2,2),activation = 'linear')
      B1 = BatchNormalization()
      R1 = ReLU()
      C2 = Conv2D(num_filter, (3,3),padding = 'same',strides = (1,1),activation = 'linear')
      B2 = BatchNormalization()
      C3 = Conv2D(num_filter, (1,1),padding= 'same', strides = (2,2),activation = 'linear')
      B3 = BatchNormalization()
      forward = B2(C2(R1(B1(C1(x))))) + B3(C3(x))
      return ReLU()(forward)
    else:
      C1 = Conv2D(num_filter, (3,3),padding = 'same',strides = (1,1),activation = 'linear')
      B1 = BatchNormalization()
      R1 = ReLU()
      C2 = Conv2D(num_filter, (3,3),padding = 'same',strides = (1,1),activation = 'linear')
      B2 = BatchNormalization()
      forward = B2(C2(R1(B1(C1(x))))) + x
      return ReLU()(forward)


  def call(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.RL(x)
    # BasicBlock1
    x = self.make_BB(x,64,False)
    x = self.make_BB(x,64,False)
    # BasicBlock2 
    x = self.make_BB(x,128,True)
    x = self.make_BB(x,128,False)
    # BasicBlock3
    x = self.make_BB(x,256,True)
    x = self.make_BB(x,256,False)
    # BasicBlock4
    x = self.make_BB(x,512,True)
    x = self.make_BB(x,512,False)

    x = self.avg(x)
    x = self.flatten(x)
    return self.FC(x)
  
  # func is just for displaying model summary.
  def func(self):
    x = tf.keras.layers.Input(shape=(32, 32, 3))
    return Model(inputs=[x], outputs=self.call(x))
    
model = MyModel()


optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(optimizer= optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
model.fit(x_train,y_train, epochs = 50)

Train on 50000 samples
Epoch 1/50
50000/50000 [==============================] - 6380s 128ms/sample - loss: 15.9479 - sparse_categorical_accuracy: 0.0100
Epoch 2/50
50000/50000 [==============================] - 6313s 126ms/sample - loss: 15.9569 - sparse_categorical_accuracy: 0.0100
Epoch 3/50
50000/50000 [==============================] - 6388s 128ms/sample - loss: 15.9569 - sparse_categorical_accuracy: 0.0100
Epoch 4/50
50000/50000 [==============================] - 6390s 128ms/sample - loss: 15.9569 - sparse_categorical_accuracy: 0.0100
Epoch 5/50
50000/50000 [==============================] - 6333s 127ms/sample - loss: 15.9569 - sparse_categorical_accuracy: 0.0100
Epoch 6/50
50000/50000 [==============================] - 6364s 127ms/sample - loss: 15.9569 - sparse_categorical_accuracy: 0.0100
Epoch 7/50
12352/50000 [======>.......................] - ETA: 1:20:05 - loss: 15.9654 - sparse_categorical_accuracy: 0.0095